 # **Covid 19 Analysis Dashboard**

In [2]:
# importing libraries
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px #The plotly.express module is plotly’s high-level API for rapid figure generation.
import folium
import plotly.graph_objects as go #The primary classes defined in the plotly.graph_objects module are Figure and an ipywidgets-compatible variant called FigureWidget, which both represent entire figures.
import seaborn as sns
import ipywidgets as widgets #interactive HTML widgets for Jupyter notebooks

In [3]:
# loading data from the source and creating required dataframes:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

## Data Frame used

In [4]:
country_df=country_df.dropna(axis=1) #dropping columns with Nan values
country_df.head(5)

,Country_Region,Last_Update,Confirmed,Deaths,Mortality_Rate,UID,Cases_28_Days,Deaths_28_Days
0,Afghanistan,2021-12-08 09:21:25,157508,7317,4.645478,4,1111,27
1,Albania,2021-12-08 09:21:25,202295,3122,1.543291,8,12170,152
2,Algeria,2021-12-08 09:21:25,211859,6114,2.885882,12,4474,160
3,Andorra,2021-12-08 09:21:25,18815,133,0.706883,20,3098,3
4,Angola,2021-12-08 09:21:25,65301,1735,2.656927,24,539,12


In [5]:
# data cleaning

# renaming the dataframe column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = recovered_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})

In [6]:
#calculating total number of confirmed cases and deaths
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())


## Total World Stats

In [7]:
# displaying the total stats using basic html code

#span style is used to color a part of text
display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: yellow; font-size:35px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: maroon; font-size:35px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "</div>")
       )

## Worst hit countries (sorted on the basis of total confirmed cases)
### *Change value of n to get the required number of countries* 

In [8]:
# in this cell we are sorting the values of confirmed cases in descending order and then displaying an interactive table
import ipywidgets
from ipywidgets import widgets
from IPython.display import display
fig = go.FigureWidget( layout=go.Layout() ) #Here the FigureWidget class of plotly.graph_objects module is used (compatible with ipywidgets)
def highlight_col(x): # function to set the colors of the columns to be highlighted 
    r = 'background-color: red'
    y = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 2] = y
    df1.iloc[:, 3] = r
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')# interact function defined in ipywidgets module used
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

#Change the value of n below to interact with this table of the dashboard

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

## Worst hit countries (sorted on the basis of total confirmed cases)
### *Use slider to get the required number of countries* 

In [9]:
def bubble_chart(n):#function to get scatter plot using plotly express (to be passed to the interact funcion)
    sorted_country_df=country_df.sort_values('confirmed', ascending= False).head(n)#sorting df on the basis of total confirmed cases
    fig = px.scatter(sorted_country_df, x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)#creating figure using px.scatter command
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    ) #Formatting the figure
    fig.show();

interact(bubble_chart, n=10) #using inbuilt interact  function of ipywidgets library to make the graph interactive 

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)
#interactive dashboard chart 2, use the slider to get the first n worst hit countries

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

## Confirmed and Death cases timeline
### *Type in the country name whose information you want* 

In [10]:
import plotly.graph_objects as go
def plotfunc(country):
    labels = ['confirmed', 'deaths']
    colors = ['yellow', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    df_list = [confirmed_df, death_df]
    fig = go.Figure();#using Figure class of plotly.graph_objects module to get access to various inbuilt functions defined in it
    
    for i, df in enumerate(df_list): #Enumerate() method adds a counter to an iterable and returns it in a form of enumerating object. i is the counter and df_list is the iterable
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,5:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,5:]),axis = 0)
        
        #to add tracing information as you move along the line graph
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
             name=labels[i],
             line=dict(color=colors[i], width=line_size[i]),
             connectgaps=True,
             text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ))

    fig.show();
interact(plotfunc,country='World');

#Below you can type the country whose data you want(Eg-Japan,China,India)

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

## Top 10 Worst hit countries (sorted on the basis of total deaths)

In [11]:
sorted_country_df=country_df.sort_values('deaths', ascending= False).head(10)
#using px.bar function of plotly express to get an iteractive bar chart
px.bar(
    sorted_country_df.head(10),#getting top 10 worst hot countries
    x = "country", #x axis title
    y = "deaths", #y axis title
    title= "Top 10 worst affected countries (based on deaths)", # Setting title of bar graph
    color_discrete_sequence=["maroon"], #setting color
    height=500, #setting image dimensions
    width=800   #setting image dimensions
) 

In [12]:
#after installing all required libraries given on top we need to install the voila library to render the dashboard on the new webpage
#use voila PythonNotebookpath  command to render the dashboard (in the anaconda prompt)
#or use the voila button in jupyter notebook (after installing voila and trusting jupyter notebook) to render the dashboard